In [68]:
from tortus import Tortus
import pandas as pd
import os

In [77]:
def annotate_tweets(path, text_column = "cleaned_content", num_records=100, prev_annotations = None, additional_labels = []):
    df = pd.read_csv(path)
    output_path = f"{os.path.splitext(path)[0]}_annotations.csv"
    if os.path.exists(output_path):
        print(f"Annotations already exist for: {output_path}, adding to these annotations")
        prev_annotations = pd.read_csv(output_path, index_col = "Unnamed: 0")
    tortus = Tortus(df, text_column, num_records=num_records, annotations=prev_annotations, labels=["standard_english", "not_standard_english", "not_english"] + additional_labels)
    tortus.annotate()
    return tortus, output_path

def save_annotations(tortus, output_path):
    tortus.annotations.to_csv(output_path)


In [79]:
tortus, output_path = annotate_tweets("data/Singapore/GenerateMostRecentTweets/3000_most_recent_tweets/0.4_to_0.5_english_words.csv", num_records = 10)

Output()

In [80]:
save_annotations(tortus, output_path)
tortus.annotations

,id_column,cleaned_content,label,annotated_at
Unnamed: 0,,,,
0,68,alles autos credit ad 2 colour copyrighted amp...,standard_english,2022-10-03 17:02:22
1,1,paling best texture dia cair macam toner tapi ...,not_english,2022-10-03 17:02:25
2,26,tak gak amik yang kiwi passionfruit guava tu s...,not_english,2022-10-03 17:02:26
40,13,poling jebakan yang paling gak pantas katanya,not_english,2022-10-03 17:03:08
4,61,hmm nike tak nk supply boot kat aliksan dgn sp...,not_english,2022-10-03 17:02:30
5,48,arghh nak nangis nak top 5 dapat top 4 jomm ki...,not_english,2022-10-03 17:02:32
6,29,sb se pehly tmhain doob k mar jana chaiye bc,not_english,2022-10-03 17:02:34
7,36,hyderabadi humour uno cycle chalate chalate pa...,not_english,2022-10-03 17:02:36
8,60,dipimpin nagita slavina inilah profil es teh i...,not_english,2022-10-03 17:02:38


In [76]:
tortus.annotations

,Unnamed: 0,cleaned_content,label,annotated_at
id_column,,,,
68,0,alles autos credit ad 2 colour copyrighted amp...,standard_english,2022-10-03 17:02:22
1,1,paling best texture dia cair macam toner tapi ...,not_english,2022-10-03 17:02:25
26,2,tak gak amik yang kiwi passionfruit guava tu s...,not_english,2022-10-03 17:02:26
13,40,poling jebakan yang paling gak pantas katanya,not_english,2022-10-03 17:03:08
61,4,hmm nike tak nk supply boot kat aliksan dgn sp...,not_english,2022-10-03 17:02:30
48,5,arghh nak nangis nak top 5 dapat top 4 jomm ki...,not_english,2022-10-03 17:02:32
29,6,sb se pehly tmhain doob k mar jana chaiye bc,not_english,2022-10-03 17:02:34
36,7,hyderabadi humour uno cycle chalate chalate pa...,not_english,2022-10-03 17:02:36
60,8,dipimpin nagita slavina inilah profil es teh i...,not_english,2022-10-03 17:02:38
